## Playlist recommendation system
1. Iskanje podobnih playlistov (KNN mean) - izberemo 3 njabolj podobne
2. Izbira najbolj bliznjih/podobnih pesmi (KNN) - izberemo 2 pesmi iz vsakega playlista
3. Predlagamo 6 najbolj podobnih pesmi (3 PL * 2 pesmi)

Playliste si lahko predstavljamo kot uporabnike, najprej izmed vseh uporabnikov izberemo najbolj podobne, in nato iz vsakega vzamemo po 2 najbolj podbni pesmi in jih predlagamo uporabniku.

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
def playlist(df_pl):
    # df_pl.drop_duplicates(subset=['id'], inplace=True)
    # df_pl.set_index("id", inplace=True)

    df_audio = df_pl.loc[:, ['danceability', 'energy', 'loudness', 
                      'speechiness', 'acousticness', 'instrumentalness',
                       'liveness', 'valence', 'tempo', 'duration_ms']]
    
    return df_audio, df_pl

In [3]:
def playlists(path):
    pls = []
    df = pd.read_csv(path, index_col=0)
    i = 0
    df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    while df_pl.shape[0] > 0:
        pls.append(df_pl)
        i += 1
        df_pl = pd.DataFrame(df.loc[df['playlist'] == i])
    
    tab_audio = []
    tab_df = []
    
    for pl in pls:
        df_audio, df_pl = playlist(pl)
        tab_audio.append(df_audio)
        tab_df.append(df_pl)
         
    return [tab_df, tab_audio]

In [4]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [5]:
tab_df_pl = playlists('Data\Playlist\Playlists5.csv')
tab_df = tab_df_pl[0]
audio_df = tab_df_pl[1]

In [6]:
len(tab_df_pl[0])

24

In [7]:
tab_df[0].head(1)

,duration_ms,tempo,loudness,name,artist_id,artist,album_id,album,popularity,playlist,danceability,energy,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,time_signature
id,,,,,,,,,,,,,,,,,,,,
17i5jLpzndlQhbS4SrTd0B,0.081501,0.343939,0.578072,"Cello Suite No. 1 in G Major, BWV 1007: I. Pré...",5aIqB5nVVvmFsvSdExz408,Johann Sebastian Bach,3cvjMNWg7t46NyDZzujfNg,"Bach: Cello Suites Nos. 1, 5 & 6",65,0,0.275,0.157,7,1,0.0636,0.89,0.842,0.186,0.304,4


In [8]:
audio_df[0].head(5)

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
id,,,,,,,,,,
17i5jLpzndlQhbS4SrTd0B,0.2750,0.1570,0.578072,0.0636,0.890,0.842,0.186,0.3040,0.343939,0.081501
4SFBV7SRNG2e2kyL1F6kjU,0.2210,0.1260,0.423640,0.0447,0.989,0.897,0.102,0.2160,0.627115,0.070859
3zLTPuucd3e6TxZnu2dlVS,0.2890,0.0306,0.299563,0.0446,0.987,0.911,0.102,0.1180,0.589478,0.130546
7h6GoPvGHC9uzZJ8bNvfIq,0.0753,0.0700,0.380955,0.0440,0.918,0.947,0.146,0.0625,0.374500,0.256123
5x3TUfYzgZIdt0Kaop64ss,0.1300,0.1580,0.638688,0.0350,0.748,0.924,0.100,0.0998,0.399044,0.204397


In [9]:
def playlist_similar(pl, tab_df_pl):
    pl = tab_df_pl[1][pl]
    tab_audio = tab_df_pl[1]
    tab_distances = []
    for index, nbr in enumerate(tab_audio):
        nbrs = NearestNeighbors(n_neighbors=1).fit(pl)
        distances, indices = nbrs.kneighbors(nbr)
        tab_distances.append((distances.mean(), index))
        
    return tab_distances

S funkcijo 'playlist_similar' poiscemo playliste, ki so najbolj podobni playlist, za katero napovedujemo nove pesmi. 

S KNN metodo poiscemo najbloj podobne sosede nekega playlista in povprecimo njihove razdalje. Tako dibimo povprecno razdaljo (podobnost) do nekega playlista. To moramo preveriti za vse playliste, ki jih imamo.

In [10]:
pl = 0
distances = playlist_similar(pl, tab_df_pl)

Za playlist 0 poiscemo najbolj podobne sosede (metoda KNN - najbliznji sosedje, razvrstimo po povprecni razdalji). Na koncu izberemo prve 3.

In [11]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [12]:
# 3 najbljznji sosedni playlisti
print("Playlist 0")
print()
print("Najbolj podobni playlist:")
for x in sorted(distances)[1:4]:
    if(x[0] < 1):
        print("Playlist: %d (razdalja %.2f)" % (x[1], x[0]))

Playlist 0

Najbolj podobni playlist:
Playlist: 2 (razdalja 0.17)
Playlist: 1 (razdalja 0.21)
Playlist: 3 (razdalja 0.24)


Ugotovimo, da je playlist 0 najbolj podoben playlistu 1,2,3.

In [13]:
index1 = sorted(distances)[1][1]
df = audio_df[index1]
df1 = tab_df[index1]

In [14]:
print(tab_df_pl[1][0].shape) # playlist
print(df.shape) # 1st similar

(147, 10)
(137, 10)


In [15]:
nbrs = NearestNeighbors(n_neighbors=10).fit(tab_df_pl[1][0])
distances, indices = nbrs.kneighbors(df)
tab = []
for index, dist in enumerate(distances):
    tab.append((dist.mean(), index))

In [16]:
len(tab)

137

In [17]:
print("Razdalja, vrstica playlista")
sorted(tab)[:3] # 3 najbolj podobne pesmi (po razdalji)

Razdalja, vrstica playlista


[(0.13690205679960055, 13), (0.1410022734885636, 9), (0.14873970007278992, 29)]

In [18]:
for row in sorted(tab)[:3]:
    print(df1.iloc[row[1]]['name'])

Ave Maria, S. 558 (after Schubert, D. 839)
Nocturne No.1 In B Flat Minor, Op.9 No.1
Pavane, Op.50


S funkcijo 'playlist_recommendation' napovedujemo pesmi, ki menimo, da bi jih uporabnik nekega playlista poslusal. 
Napovedane pesmi so podobne pesmim iz playlista uporabnika, in so izvzete iz playlistov drugih uporabnikov (sorodnih).

Naprej poiscemo 3 najbolj podobne (sorodne) playliste, nato izberemo iz vsakega od teh playlistov 2 nabolj primerni pesmi za napoved.

In [19]:
def playlist_recommendation(pl_n, tab_df_pl):
    distances = playlist_similar(pl_n, tab_df_pl)
    pl = tab_df_pl[1][pl_n]
    print("Recommending for playlist num: %d (len: %d)" % (pl_n, pl.shape[0])) # playlist
    rec_songs = []
    
    # 3 najbljznji sosedni playlisti
    print("3 najbolj podobni playlisti:")
    for x in sorted(distances)[1:4]:
        if(x[0] < 1):
            index1 = x[1]
            df_audio = tab_df_pl[1][index1]
            df = tab_df_pl[0][index1]
            print('n%d: %.3f (len: %d)' % (x[1], x[0], df.shape[0]))
            
            # df_audio - najblolj podobni (similar) playlisti
            # pl - playlist, za katerega napovedujemo pesmi 
            
            # za vsako pesem iz podobnega playlista (df_audio), poiscemo 10 najbliznjih (podobnih) pesmi v playlistu -  pl
            # izracunamo povprecno razdaljo (manjsa je razdalja bolj je pesem podobna pesmim na playlistu - pl)
            # izberemo 3 najbolj podobne pesmi
            nbrs = NearestNeighbors(n_neighbors=10).fit(pl)
            distances1, indices = nbrs.kneighbors(df_audio)
            tab = []
            for index, dist in enumerate(distances1):
                tab.append((dist.mean(), index))
                
            # 2 najbolj podobne pesmi (po razdalji)
            for row in sorted(tab)[:2]:
                rec_songs.append(df.iloc[row[1]])
                #print(df.iloc[row[1]]['name'])
                
    return rec_songs

In [20]:
 # PLAYLISTS
    # 0,1,2,3 - classical
    # 4,5,6,7 - rap
    # 8,9 - edm
    # 10 - techno
    # 11 - House
    # 12,13,14,15 - rock
    # 16,17,18,19 - latin
    # 20,21,22,23 - pop

In [21]:
pl = 18
rec_songs = playlist_recommendation(pl, tab_df_pl)

Recommending for playlist num: 18 (len: 85)
3 najbolj podobni playlisti:
n19: 0.184 (len: 79)
n16: 0.213 (len: 100)
n17: 0.224 (len: 50)


In [22]:
print("Recommended songs for playlist: ", pl, '\n')

for song in rec_songs:
    print("Name: ", song['name'], "|| Artist: ", song['artist'])

Recommended songs for playlist:  18 

Name:  Caraluna || Artist:  Bacilos
Name:  Mi primer millon || Artist:  Bacilos
Name:  Ley Seca || Artist:  Jhay Cortez
Name:  Me Enamora || Artist:  Mau y Ricky
Name:  MAMIII || Artist:  Becky G
Name:  X ÚLTIMA VEZ || Artist:  Daddy Yankee


UGOTOVITVE: 

Sistem napovedovanja pesmi deluje vredu na izbranih playlistah (24).

Za klasicno playlisto napove klasicne pesmi, za playlisto, kjer je veliko rapa napove rap pesmi...

Playliste si lahko predstavljamo kot uporabnike, najprej izmed vseh uporabnikov izberemo najbolj podobne, in nato iz vsakega vzamemo po 2 najbolj podbni pesmi in jih predlagamo uporabniku.

Sistem zgleda da deluje vredu, uporabniku, ki poslusa rap napove pesmi rapa... Seveda pa je izbira pesmi uporabnika se vedno subjektivna, in na njegovo izbira vpliva vec kot pa le 10 znacilnosti pesmi. 